In [38]:
import re
import sys
import itertools

def keymap_lookup(keymap, nums):
    ret = []
    for num in nums:
        for k, v in keymap.items():
            if k == num:
                ret.append(v)
    #print(ret)       
    return ret

def read_content(filename='words.txt'):
    with open(filename, 'r') as f:
        return f.read().strip().replace(',', ' ')

#make a dictionary mapping a word to its frequency
def parse_content(content):
    content = content.split()
    word_dict = {}
    i = 0
    #save word-frequency pairs as dict
    while(i < len(content)):
        word_dict[content[i]] = content[i + 1]
        i += 2
    #print(word_dict)
    return word_dict

def ask_for_numbers():
    while True:
        response = input('What numbers have you pressed? ').strip()
        if len(response) < 3:
            print('You need to enter at least three numbers.', file=sys.stderr)
        elif re.search("[^2-9]", response):
            print("You entered a character that isn't one of 2, 3, 4, 5, 6, 7, 8, or 9. Please try again.", file=sys.stderr)
        else:
            return response

#return a dictionary mapping letters/words 
#in a tree structure
def make_tree(words):
    tree = {'children': {}}
    for word, number in words.items():
        subdict = tree
        for letter in word:
            if letter not in subdict['children']:
                subdict['children'][letter] = {'children': {}}
            subdict = subdict['children'][letter]
        #end-of-word 
        subdict['word'] = word
        subdict['freq'] = number
    #print(tree)
    return tree
        
#return all nodes with words that have given prefix (as dict)
def search(tree, prefix):
    subdict = tree
    for letter in prefix:
        if letter in subdict['children']:
            subdict = subdict['children'][letter]
        else:
            return {'children': {}}
    return subdict

def flatten_dict(subdict):
    res =[]
    if 'word' in subdict:
        res.append((subdict['word'], subdict['freq']))
    for item in subdict['children'].values():
        res.extend(flatten_dict(item))
    return res

#look up words for prefixes from T9 numbers entered by user
def predict(tree, numbers, keymap):
    #get possible letters for numbers
    t9_letters = keymap_lookup(keymap, numbers)
    #get all possible combinations
    combis = list(itertools.product(*t9_letters)) #itertools.product() is used to find the cartesian product from the given iterator
    
    res = []
    #search the tree for the prefixes
    for c in combis:
        node = search(tree, c)
        res.extend(flatten_dict(node))
    return sorted(res, key=lambda x: x[1], reverse=True)

In [ ]:
keymap = {
    '2': 'abc',
    '3': 'def',
    '4': 'ghi',
    '5': 'jkl',
    '6': 'mno',
    '7': 'pqrs',
    '8': 'tuv',
    '9': 'wxyz',
}

content = read_content(filename='test.txt')

words = parse_content(content)

tree = make_tree(words)
# print(tree)
# predictions = predict(tree, '226', keymap)
# print(predictions)
#print(list(map(flatten_dict, predictions)))

while True:
    numbers = ask_for_numbers()
    predictions = predict(tree, numbers, keymap)

    if not predictions:
        print('No words were found that match those numbers. :(')
    else:
        for prediction in predictions:
            print(prediction)

    answer = input('Do you want to go again? [y/n]\n')
    again = answer and answer[0] in ('y', 'Y')
    if not again:
        break

What numbers have you pressed? 226
('ban', '234')
('band', '45')
('candy', '19')
('can', '1')


In [ ]:
words = {
  'ban': 10,
  'band': 5,
  'bar': 14,
  'can': 32,
  'candy': 7
}

{
   "b": {
      "a": {
         "n": {
            "$ban": 10,
            "d": {
               "$band": 5
            }
         },
         "r": {
            "$bar": 14
         }
      }
   },
   "c": {
      "a": {
         "n": {
            "$can": 32,
            "d": {
               "y": {
                  "$candy": 7
               }
            }
         }
      }
   }
}


{
    "freg": 10,
    'word':"ban",
    'children':{
        "d": {
            "$band": 5
        } 
    }

}